In [1]:
from onnxruntime.transformers.convert_generation import parse_arguments, GenerationType, convert_generation_model
from models.gpt2.convert_to_onnx import main as convert_gpt2_to_onnx

/Users/esp.py/Projects/Personal/end-to-end-rag/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [30]:
from pathlib import Path

model_path = Path.cwd().joinpath('models')
model_id = 'bio-gpt-qa'
model_path = model_path.joinpath(model_id)

In [3]:
onnx_path = Path.cwd().joinpath('models_repository',
                                "generator", "generator_model", "1", )

In [4]:
onnx_model_with_past = onnx_path.joinpath('bio-gpt-model-with-past')

In [5]:
from argparse import Namespace

In [6]:
arguments = Namespace(
    model_name_or_path=model_path.__str__(),
    output=onnx_model_with_past.parent.joinpath(
        "biogpt-model-with-past-and-beam.onnx").__str__(),
    model_type="gpt2",
    num_beams="5",
    temperature="0.25",
    model_class="BioGptModel"
)

In [7]:
onnx_model_with_past.joinpath("model.onnx").exists()

True

In [8]:
arguments_list = []

for key, value in vars(arguments).items():
    arguments_list.append(f"--{key}")
    arguments_list.append(value)

In [9]:
arguments_list

['--model_name_or_path',
 '/Users/esp.py/Projects/Personal/end-to-end-rag/models/bio-gpt-qa',
 '--output',
 '/Users/esp.py/Projects/Personal/end-to-end-rag/models_repository/generator/generator_model/1/biogpt-model-with-past-and-beam.onnx',
 '--model_type',
 'gpt2',
 '--num_beams',
 '5',
 '--temperature',
 '0.25',
 '--model_class',
 'BioGptModel']

In [10]:
args = parse_arguments(arguments_list)

In [11]:
convert_generation_model(args=args, generation_type=GenerationType.BEAMSEARCH)

Arguments:Namespace(model_name_or_path='/Users/esp.py/Projects/Personal/end-to-end-rag/models/bio-gpt-qa', model_class='BioGptModel', cache_dir='./cache_models', output='/Users/esp.py/Projects/Personal/end-to-end-rag/models/bio-gpt-qa_past_fp32.onnx', optimize_onnx=True, use_gpu=False, provider=None, tolerance=0.0005, input_test_file='', precision=<Precision.FLOAT32: 'fp32'>, test_cases=10, test_runs=1, verbose=False, use_external_data_format=False, overwrite=True, use_int64_inputs=False, stage=0, auto_mixed_precision=False, keep_io_types=False, io_block_list=[], op_block_list=[], node_block_list=[], force_fp16_initializers=False)
PyTorch Version:2.2.0
Transformers Version:4.38.2
Onnxruntime Version:1.15.1


usign the model class <class 'gpt2_helper.MyBioGptModel'>


Exporting ONNX model to /Users/esp.py/Projects/Personal/end-to-end-rag/models/bio-gpt-qa_BioGptModel_past.onnx
Shapes: input_ids=torch.Size([1, 1]) past=torch.Size([2, 1, 16, 1, 64]) output=torch.Size([1, 1, 42393]) present=torch.Size([2, 1, 16, 2, 64])
/Users/esp.py/Projects/Personal/end-to-end-rag/.venv/lib/python3.10/site-packages/transformers/models/biogpt/modeling_biogpt.py:495: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  elif attention_mask.shape[1] != past_key_values_length + input_shape[1]:
/Users/esp.py/Projects/Personal/end-to-end-rag/.venv/lib/python3.10/site-packages/transformers/modeling_attn_mask_utils.py:114: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so 

Testing the model for the inference session

In [15]:
model_with_beam = onnx_model_with_past.parent.joinpath(
        "biogpt-model-with-past-and-beam.onnx").__str__()

In [14]:
from onnxruntime import InferenceSession

In [19]:
import torch

In [17]:
inference_session = InferenceSession(model_with_beam, providers=['CPUExecutionProvider'])

2024-03-05 11:25:16.375628 [W:onnxruntime:, graph.cc:3543 CleanUnusedInitializersAndNodeArgs] Removing initializer '/biogpt/Shape_4_output_0'. It is not used by any node and should be removed from the model.
2024-03-05 11:25:16.375654 [W:onnxruntime:, graph.cc:3543 CleanUnusedInitializersAndNodeArgs] Removing initializer '/biogpt/Constant_8_output_0'. It is not used by any node and should be removed from the model.
2024-03-05 11:25:16.404403 [W:onnxruntime:, graph.cc:3543 CleanUnusedInitializersAndNodeArgs] Removing initializer '/Constant_output_0'. It is not used by any node and should be removed from the model.


Yeah, it worked.. 

Generating inputs

In [23]:
from typing import Sequence

In [26]:
from transformers import AutoTokenizer, AutoConfig

In [43]:
EXAMPLE_Text = ["what is the cause of covid-19?"]


def get_tokenizer(model_name_or_path: str, cache_dir: str):
    tokenizer = AutoTokenizer.from_pretrained(
        model_name_or_path, cache_dir=cache_dir)
    tokenizer.padding_side = "left"
    tokenizer.pad_token = tokenizer.eos_token
    return tokenizer

In [45]:
model_path

PosixPath('/Users/esp.py/Projects/Personal/end-to-end-rag/models/bio-gpt-qa')

In [46]:
model_config = AutoConfig.from_pretrained(model_path.__str__())

In [47]:
tokenizer = get_tokenizer(model_path, model_path)
encodings_dict = tokenizer.batch_encode_plus(EXAMPLE_Text, padding=True)

input_ids = torch.tensor(encodings_dict["input_ids"], dtype=torch.int32)

In [48]:
input_ids

tensor([[    2,  1994,    21,     6,   533,     5,  1181, 17270,     9,   656,
           927]], dtype=torch.int32)

In [49]:
empty_past

[tensor([], size=(2, 1, 16, 0, 64)),
 tensor([], size=(2, 1, 16, 0, 64)),
 tensor([], size=(2, 1, 16, 0, 64)),
 tensor([], size=(2, 1, 16, 0, 64)),
 tensor([], size=(2, 1, 16, 0, 64)),
 tensor([], size=(2, 1, 16, 0, 64)),
 tensor([], size=(2, 1, 16, 0, 64)),
 tensor([], size=(2, 1, 16, 0, 64)),
 tensor([], size=(2, 1, 16, 0, 64)),
 tensor([], size=(2, 1, 16, 0, 64)),
 tensor([], size=(2, 1, 16, 0, 64)),
 tensor([], size=(2, 1, 16, 0, 64)),
 tensor([], size=(2, 1, 16, 0, 64)),
 tensor([], size=(2, 1, 16, 0, 64)),
 tensor([], size=(2, 1, 16, 0, 64)),
 tensor([], size=(2, 1, 16, 0, 64)),
 tensor([], size=(2, 1, 16, 0, 64)),
 tensor([], size=(2, 1, 16, 0, 64)),
 tensor([], size=(2, 1, 16, 0, 64)),
 tensor([], size=(2, 1, 16, 0, 64)),
 tensor([], size=(2, 1, 16, 0, 64)),
 tensor([], size=(2, 1, 16, 0, 64)),
 tensor([], size=(2, 1, 16, 0, 64)),
 tensor([], size=(2, 1, 16, 0, 64))]

In [52]:
import numpy as np

In [79]:
ort_inputs = {
    "input_ids": np.ascontiguousarray(input_ids.cpu().numpy(), dtype=np.int32),
    "max_length": np.array([512], dtype=np.int32),
    "min_length": np.array([128], dtype=np.int32),
    "num_beams": np.array([5], dtype=np.int32),
    "num_return_sequences": np.array([1], dtype=np.int32),
    "length_penalty": np.array([1.0], dtype=np.float32),
    "repetition_penalty": np.array([1.4], dtype=np.float32),
}

In [80]:
outputs = inference_session.run(None, ort_inputs)

In [87]:
output_indices = outputs[0].squeeze()

In [88]:
tokenizer = get_tokenizer(model_path.__str__(), model_path.__str__())

In [89]:
tokenizer.decode(output_indices, skip_special_tokens=True)

'what is the cause of covid-19? context: The disease is known to have an impact on morbi-mortality. We present a clinical case study of a 60-year-old female, with a history of hypertension, who was admitted to the hospital due to cough, sore throat, fever and weight loss. The patient had a medical history of hypertension, in which she had been treated for several years, with poor control of blood pressure. The physical examination revealed a cervical adenopathy, without any other abnormalities. The chest x-ray showed a mediastinal mass, which was confirmed by a biopsy of the cervical lymph nodes. The patient was diagnosed with primary mediastinal large B-cell lymphoma, stage IV. The patient was treated with chemotherapy, but the disease did not improve until she died 7 months later. <unk><unk><unk><unk><unk><unk><unk><unk><unk>the answer to the question given the context is yes.'

the code seems to be working, we can continue with the deployment and write the blogpost.